## Jpype库详解
　　参考：[python安装Jpype](https://blog.csdn.net/weixin_39598206/article/details/88665502)<br>

### 1、简介
　　JPype是一个能够让 python 代码方便地调用 Java 代码的工具，从而克服了 python 在某些领域（如服务器端编程）中的不足。<br>
　　JPype的实际运行环境仍然是python runtime，只是在运行期间启动了一个嵌入的jvm。<br>
  
### 1、报错处理
#### 1、关于JPype报FileNotFoundError: [Errno 2] No such file or directory: '/usr/lib/jvm'错误的解决
　　找不到jvm，大概原因是环境变量没有生效。<br>
　　平时在命令行中能直接运行java，是因为java所在的bin目录被添加到PATH，且由export PATH后作为环境变量生效。但是JAVA_HOME只是作为普通变量，使用os.getenv()的时候获取环境变量时找不到JAVA_HOME，所以推测应该只要将JAVA_HOME前面添加export，重启项目即可。<br>
　　**解决办法：**<br>
　　　　在全局配置文件/etc/profile或个人配置文件~/.bashrc或~/.bash_profile中添加export JAVA_HOME即可，如下是我的/etc/profile的设置：<br>
># set java path <br>
export JAVA_HOME=/usr/local/java/latest<br>
export PATH=\\$\{JAVA_HOME\}/bin:\\${PATH}<br>
export CLASSPATH=.:\\$JAVA_HOME/lib/dt.jar:\\$JAVA_HOME/lib/tools.jar<br>
其他注意事项：如果使用PyCharm远程调试，若遇到同样报错，需要在导入pyhanlp前先设置环境变量，如下<br>
<br>
\# 设置环境变量<br>
import os<br>
os.environ\['JAVA_HOME'\] = '/usr/local/java/latest'<br>
<br>
\# 再导入JPype 即可避免无法找到java的问题<br>
import jpype<br>

In [1]:
import jpype


# 指定jar包位置, 或者.class文件
# class_path = 'E:/SinaifMock/utils/Test.class'
jar_path = "E:\create_TestAccount/test.jar"

# JVM的路径位置
JVM_path = "E:/java/jdk1.8.0_121/jre/bin/server/jvm.dll"

# 开启JVM，且指定jar包, 或者.class文件位置
jpype.startJVM(JVM_path, jar_path)

# 打印hello, word
jpype.java.lang.System.out.println("hello World")

# 引入java程序中的类.路径应该是项目中的package包路径.类名
javaClass = jpype.JClass('Test')

# 创建一个对象
javaInstance = javaClass()
# 执行类中的函数了
num = javaInstance.getSum(10, 20)
string = javaInstance.getString("getString")
print(num, string)

# 关闭JVM
jpype.shutdownJVM()

ModuleNotFoundError: No module named 'jpype'